In [3]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time

import numpy as np
import matplotlib.pyplot as plt
import os

/Users/apple/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/apple/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <08E12B12-6183-307E-BDA0-374FA8EBA2C9> /Users/apple/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <57D24B07-8B24-3888-A2B5-2B4C95434BA4> /Users/apple/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
pip install torchvision

     |████████████████████████████████| 1.5 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 143.4 MB 730 kB/s eta 0:00:01████████████████▌            | 87.2 MB 754 kB/s eta 0:01:15
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
Note: you may need to restart the kernel to use updated packages.


In [17]:
batch_size = 16
num_epochs = 300

dataset = "/Users/apple/Desktop/Palm Print Database/Palmprint"
train_directory = os.path.join(dataset, "train")
valid_directory = os.path.join(dataset, "valid")

In [18]:
image_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels = 3),
        transforms.RandomResizedCrop(size = 256, scale = (0.8, 1.0)),
        transforms.RandomRotation(degrees = 15),
        transforms.CenterCrop(size = 224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Grayscale(num_output_channels = 3),
        transforms.Resize(size = 256),
        transforms.CenterCrop(size = 224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [19]:
data = {
    'train': datasets.ImageFolder(root = train_directory, transform = image_transforms['train']),
    'valid': datasets.ImageFolder(root = valid_directory, transform = image_transforms['valid'])
}

In [23]:
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])

train_data = DataLoader(data['train'], batch_size = batch_size, shuffle = True)
valid_data = DataLoader(data['valid'], batch_size = batch_size, shuffle = True)

In [24]:
print(train_data_size, valid_data_size)

297 297


In [33]:
resnet18 = models.resnet18(pretrained = True)
for param in resnet18.parameters():
    param.requires_grad = False
fc_inputs = resnet18.fc.in_features
resnet18.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 99),
    nn.LogSoftmax(dim = 1)
)

resnet18 = resnet18.to(torch.device('cpu'))
loss_func = nn.NLLLoss()
optimizer = optim.Adam(resnet18.parameters())

In [45]:
def train_and_valid(model, loss_function, optimizer, epochs = 25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    history = []
    best_acc = 0.0
    best_epoch = 0
    
    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch + 1, epochs))
        
        model.train()
        
        train_loss = 0.0
        train_acc = 0.0
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in enumerate(train_data):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            
            loss = loss_function(outputs, labels)
            
            loss.backward()
            
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            train_acc += acc.item() * inputs.size(0)
        with torch.no_grad():
            model.eval()
            
            for j, (inputs, labels) in enumerate(valid_data):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = model(inputs)
                
                loss = loss_function(outputs, labels)
                
                valid_loss += loss.item() * inputs.size(0)
                
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))
                
                acc = torch.mean(correct_counts.type(torch.FloatTensor))
                
                valid_acc += acc.item() * inputs.size(0)
                
            avg_train_loss = train_loss / train_data_size
            avg_train_acc = train_acc / train_data_size
            
            avg_valid_loss = valid_loss / valid_data_size
            avg_valid_acc = valid_acc / valid_data_size
            
            history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
            
            if best_acc < avg_valid_acc:
                best_acc = avg_valid_acc
                best_epoch = epoch + 1
            
            epoch_end = time.time()
            
            print(
                "Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
                    epoch + 1, avg_train_loss, avg_train_acc * 100, avg_valid_loss, avg_valid_acc * 100,
                    epoch_end - epoch_start
                )
            )
            print("Best Accuracy for Validation: {:.4f} at Epoch {:03d}".format(best_acc, best_epoch))
            
            model_path = "models_resnet18_ep" + str(num_epochs)
            if not os.path.exists(model_path):
                os.makedirs(model_path)
            torch.save(model, model_path + '/' + 'Palmprint' + '_model_' + str(epoch + 1) + '.pt')
            
    return model, history, best_acc, best_epoch

In [46]:
trained_model, history, best_acc, best_epoch = train_and_valid(resnet18, loss_func, optimizer, num_epochs)

Epoch: 1/300
Epoch: 001, Training: Loss: 4.4545, Accuracy: 4.0404%, 
		Validation: Loss: 4.2921, Accuracy: 7.4074%, Time: 14.3869s
Best Accuracy for Validation: 0.0741 at Epoch 001
Epoch: 2/300
Epoch: 002, Training: Loss: 4.3673, Accuracy: 3.7037%, 
		Validation: Loss: 4.2195, Accuracy: 14.8148%, Time: 14.3306s
Best Accuracy for Validation: 0.1481 at Epoch 002
Epoch: 3/300
Epoch: 003, Training: Loss: 4.2618, Accuracy: 3.7037%, 
		Validation: Loss: 4.0815, Accuracy: 13.4680%, Time: 15.7767s
Best Accuracy for Validation: 0.1481 at Epoch 002
Epoch: 4/300
Epoch: 004, Training: Loss: 4.1534, Accuracy: 9.0909%, 
		Validation: Loss: 3.9656, Accuracy: 16.8350%, Time: 14.2091s
Best Accuracy for Validation: 0.1684 at Epoch 004
Epoch: 5/300
Epoch: 005, Training: Loss: 3.9993, Accuracy: 9.7643%, 
		Validation: Loss: 3.8405, Accuracy: 17.1717%, Time: 14.2921s
Best Accuracy for Validation: 0.1717 at Epoch 005
Epoch: 6/300
Epoch: 006, Training: Loss: 3.9569, Accuracy: 11.1111%, 
		Validation: Loss: 3

In [47]:
model_path = 'models_resnet18_ep' + str(num_epochs)
torch.save(history, model_path + '/' + 'Palmprint' + '_history.pt')
history = np.array(history)
plt.plot(history[:, 0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0, 8)
plt.title('best_acc:' + str(best_acc)[0:6] + ' best_epoch' + str(best_epoch))
plt.savefig('Palmprint' + model_path + '_loss_curve.png')
plt.close()

plt.plot(history[:, 2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.title('best_acc:' + str(best_acc)[0:6] + ' best_epoch' + str(best_epoch))
plt.savefig(dataset + model_path + '_accuracy_curve.png')
plt.close()
plt.show()